# 디스크립터 2일차

## 디스크립터 복습

In [ ]:
class descriptor:
    # self : 디스크립터 객체
    # instance : 디스크립터를 사용하는 클래스의 객체
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value
        
    def __init__(self, value):
        self.value = value

class A:
    #data = 10
    data = descriptor(10)

In [ ]:
a = A()
# a.data.value
a.data

In [ ]:
# == a.data
type(a).__dict__['data'].__get__(a, type(a))

## 실습

### 숫자를 저장하는 디스크립터 만들기

1. `__get__`, `__set__`, `__delete__` 구현
1. `__set__`을 통해 숫자를 저장한다.
1. 숫자는 호출할때마다 +1 증가
1. 숫자를 할당할때마다, 이전값에 더 합니다.(덮어쓰기 x)
1. 숫자를 지울떄는 최근에 추가한 수만큼만 제거합니다. (추가한 숫자들의 목록을 별도 저장)
1. 다음과 같이 작동해야합니다.

```
class A:
    number = descriptor()
    
    
a = A()
a.number = 1
print(a.number) # 2
print(a.number) # 3
a.number = 3
print(a.number) # 7

del a.number # -3 -> 4
print(a.number) # 5

b = A()
b.number = 1
print(b.number) # 7
```

In [ ]:
class descriptor:
    # instance -> class A 인스턴스
    # self -> 디스크립터 클래스 인스턴스
    def __get__(self, instance, owner):
        instance.value += 1
        return instance.value
    
    def __set__(self, instance, value):
        if not hasattr(self, 'value'):
            instance.value = 0
            instance.recent = []
        instance.value += value
        instance.recent.append(value)
        
    def __delete__(self, instance):
        instance.value -= instance.recent[-1]
        instance.recent.pop()
        # 최근 추가된 값만큼만 마이너스(-)
        # 그리고 최근 추가된 리스트에서 제거
    
# 함정이 있습니다. 그 함정도 수정
## > hint : 저장되는 위치

class A:
    number = descriptor()
    
a = A()
b = A()
a.number = 1
print(a.number)
a.number = 10
b.number = 3
print(b.number)

# 데이터 디스크립터  

> `__set__`, `__delete__`중 하나라도 구현이 되어있으면 


# 논데이터 디스크립터

> `__get__` 만 구현되어 있다면

In [ ]:
from inspect import isfunction, isdatadescriptor

In [ ]:
isfunction(lambda x : x)

In [ ]:
class ND:
    def __get__(self, instance, owner):
        pass


class DD:
    def __get__(self, instance, owner):
        pass
    
    def __set__(self, instance, onwer):
        pass

In [ ]:
nd = ND()
dd = DD()

In [ ]:
isdatadescriptor(nd), isdatadescriptor(dd)

# 속성 탐색

instance 속성 vs  class 속성

## 인스턴스

In [ ]:
class A:
    x = 3

In [ ]:
A.__dict__

In [ ]:
a = A()

In [ ]:
A.__dict__, a.__dict__

In [ ]:
a.x = 123

In [ ]:
a.__dict__

In [ ]:
A.__dict__

In [ ]:
a.x

# 확인 : 데이터 디스크립터는 인스턴스의 dict보다 우선 탐색된다.

In [ ]:
class data:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value

    
class A:
    data = data(3)

In [ ]:
a = A()

In [ ]:
a.data

In [ ]:
a.__dict__

In [ ]:
a.__dict__['data'] = 9

In [ ]:
a.data, a.__dict__

# 확인 2 : 논데이터 디스크립터는 인스턴스의 dict보다 나중에 탐색

In [ ]:
class nondata:
    def __init__(self):
        self.value = 3
        
    def __get__(self, instance, owner):
        return self.value
    

class A:
    nd = nondata()

In [ ]:
a = A()

In [ ]:
a.nd, a.__dict__

In [ ]:
a.__dict__['nd'] = 100

In [ ]:
a.nd

In [ ]:
type(a).__dict__['nd'].__get__(a, type(a))

# 확인 3 : `__get__`이 없는 data descriptor는 본인을 리턴한다

In [ ]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __repr__(self):
        return '< HI >'

class A:
    data = DD(3)

In [ ]:
a = A()

In [ ]:
a.data

In [ ]:
a.data.value

In [ ]:
a.data.__class__ is DD

# 확인 4 : 클래스 dict의 값이 인스턴스 dict, 데이터디스크립터 보다 늦게 탐색

In [ ]:
class A:
    def __init__(self, value):
        self.a = value
        
    a = 10

In [ ]:
a = A(3)

In [ ]:
A.__dict__, a.__dict__

In [ ]:
a.a

# 확인 5 : MRO 탐색

In [ ]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    

class A:
    pass


class B:
    value = DD(100)
    
    
class C(A, B):
    pass

In [ ]:
c = C()

In [ ]:
type(c).__mro__

In [ ]:
c.value

In [ ]:
class A:
    value = 10
    

class B:
    value = DD(100)
    

class C(A, B):
    pass

In [ ]:
c = C()
type(c).__mro__

In [ ]:
c.value

# 클래스 속성 탐색

## 메타클라스에 디스크립터 적용

In [ ]:
class md:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("md get")
        return self.value
    
    def __set__(self, instance, value):
        print('md set')
        self.value = value
        

class Meta(type):
    d = md(3)

In [ ]:
class A(metaclass=Meta):
    pass

In [ ]:
A.__dict__

In [ ]:
A.d

### 확인 6 : 메타클래스에 `__get__`이 있는 데이터 디스크립터가 클래스의 값보다 우선한다.


In [ ]:
class A(metaclass=Meta):
    d = 123

In [ ]:
A.d

# 확인 7 : 메타클래스에 `__get__`이 없는 데이터디스크립터는 클래스의 속성보다 늦다.

In [ ]:
class d1:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
        
class m(type):
    value = d1(99)

In [ ]:
class A(metaclass=m):
    value = 1

In [ ]:
A.value

# set, delete 작동

In [ ]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        raise AttributeError('NO!')
        
    def __get__(self, instance, owner):
        return self.value

class A:
    value = DD(99999)
    
    def __setattr__(self, name, value):
        print('setattr')
        super().__setattr__(name, value)

In [ ]:
a = A()

In [ ]:
a.value

In [ ]:
a.value = 123

###  클래스인 경우..

In [ ]:
class DD:
    def __init__(self, value):
        self.value = value
        
    def __set__(self, instance, value):
        raise AttributeError('NO!')
        
    def __get__(self, instance, owner):
        return self.value

In [ ]:
class M(type):
    value = DD(9123123)
    
    def __setattr__(self, name, value):
        print('class setattr')
        super().__setattr__(name, value)


In [ ]:
class A(metaclass=M):
    pass

In [ ]:
A.value

In [ ]:
A.value = 123123

### delete

In [ ]:

class DD:
    def __init__(self, value):
        self.value = value
        
    def __delete__(self, instance):
        raise AttributeError('NO!')
        
    def __get__(self, instance, owner):
        return self.value

class A:
    value = DD(99999)
    
    def __delattr__(self, name):
        print('detattr')
        super().__delattr__(name,)

In [ ]:
a = A()

In [ ]:
a.value

In [ ]:
del a.value

# 디스크립터를 사용할때, 정보를 저장하는 위치

In [ ]:
class DD:
    # self = DD Class 인스턴스
    # instance = 디스크립터를 사용하는 클래스의 인스턴스 
    def __set__(self, instance, value):
        pass
    
    def __delete__(self, instance):
        pass
    
    def __get__(self, instance, owner):
        pass
    
        
class A:
    value = DD()

## 디스크립터에 저장

In [ ]:
class descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
class A:
    value = descriptor(10)

In [ ]:
a = A()
b = A()

In [ ]:
a.value

In [ ]:
b.value

In [ ]:
a.value = 10000

In [ ]:
a.value

In [ ]:
b.value

In [ ]:
class descriptor:
    def __init__(self, value):
        self.storage = {}
        
    def __get__(self, instance, owner):
        return self.storage[instance]
    
    def __set__(self, instance, value):
        self.storage[instance] = value
        
    def __delete__(self, instance):
        del self.storage[instance]

In [ ]:
class A:
    aaa = descriptor(123)

In [ ]:
a = A()
b = A()
c = A()

In [ ]:
a.aaa = 111

In [ ]:
a.aaa

In [ ]:
b.aaa = 222

In [ ]:
c.aaa = 333

In [ ]:
a.aaa, b.aaa, c.aaa

In [ ]:
A.__dict__['aaa'].storage

클래스 A가 생성한 모든 인스턴스에서 'aaa'이름으로 지정된 속성 값들을

모두 가져올 수 있다.

# 단점

In [ ]:
del a

In [ ]:
A.__dict__['aaa'].storage

In [ ]:
import weakref

In [ ]:
wk_b = weakref.ref(b)

In [ ]:
del b

In [ ]:
wk_b

### 해결

In [ ]:
from weakref import WeakKeyDictionary

In [ ]:
dict_ = WeakKeyDictionary()

In [ ]:
class A:
    pass

In [ ]:
a = A()

In [ ]:
dict_[a] = 123

In [ ]:
dict_[a]

In [ ]:
len(dict_)

In [ ]:
del a

In [ ]:
len(dict_)

In [ ]:
from weakref import WeakKeyDictionary


class descriptor:
    def __init__(self, value):
        self.storage = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        return self.storage[instance]
    
    def __set__(self, instance, value):
        self.storage[instance] = value
        
    def __delete__(self, instance):
        del self.storage[instance]

In [ ]:
from collections import UserDict

class A(UserDict):
    d = descriptor(10)

In [ ]:
a = A()

In [ ]:
a.d = 10

## id를 사용

In [ ]:
d = {1, 2, 3}

In [ ]:
d

In [ ]:
hash(d)

In [ ]:
a = {}

In [ ]:
a[d]

In [ ]:
a[id(d)] = 123

In [ ]:
a

In [10]:
from weakref import finalize

In [17]:
class B:
    pass

b = B()

In [13]:
finalize(b, print, '제거', '합니다')

<finalize object at 0x10c614ca0; for 'B' at 0x10eaa6438>

In [14]:
del b

# 인스턴스 속성에 저장하는 디스크립터

In [ ]:
class dd:
    def __set__(self, instance, value):
        instance.data = value
        
    def __get__(self, instance, value):
        return instance.data
        
    def __delete__(self, instance):
        del instance.data

In [ ]:
class A:
    d = dd()

In [ ]:
a = A()
b = A()

In [ ]:
a.d = 123
b.d = 10

In [ ]:
a.d, b.d

### 부작용

In [ ]:
class A:
    data1 = dd()
    data2 = dd()

In [ ]:
a = A()

In [ ]:
a.data1 = 123
a.data2 = 1999999

In [ ]:
a.data1, a.data2

### 좀 더 쉽게 해결할수있는 방법이 3.6버전

`__get__`, `__set__`, `__delete__`

`__set_name__`

In [ ]:
class dd:
    def __set_name__(self, owner, name):
        self.name = name
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, value):
        return instance.__dict__[self.name]
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [ ]:
class A:
    data1 = dd()
    data2 = dd()

In [ ]:
a = A()

In [ ]:
a.data1 = 123
a.data2 = 321

In [ ]:
a.data1, a.data2

In [ ]:
a.__dict__

# 디스크립터 

### 속성검사

In [ ]:
class Validated:
    def __set_name__(self, owner, name):
        self.name = name

    def __init__(self, type_):
        self.type = type_
        
    def __set__(self, instance, value):
        if isinstance(value, self.type):
            instance.__dict__[self.name] = value
        else:
            raise AttributeError('NO')

In [ ]:
class A:
    num1 = Validated(int)
    num2 = Validated(float)

In [ ]:
a = A()

In [ ]:
a.num1 = 123
a.num2 = 1.0

In [ ]:
a.num1, a.num2

In [ ]:
a.num1 = 1.0
a.num2 = 123

## lazy

In [ ]:
class lazy:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("한번만 실행")
        value = self.func(instance)
        instance.__dict__[self.func.__name__] = value
        return value

In [ ]:
class A:
    def __init__(self, value):
        self.value = value

    @lazy
    def data(self):
        return self.value * 10

In [ ]:
a = A(111)

In [ ]:
a.data

In [ ]:
a.data

## 리드온리 

In [ ]:
class descriptor:
    def __init__(self, value):
        self.value = value
    
    def __get__(self, instance, onwer):
        return self.value

    def __set__(self, instance, value):
        raise AttributeError('No')


In [ ]:
class A:
    d = descriptor(10)

In [ ]:
a = A()

In [ ]:
a.d

In [ ]:
a.__dict__

In [ ]:
a.d = 123

In [ ]:
a.__dict__

# 상수



In [ ]:
class Constant:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        raise AttributeError('no')
        
    def __delete__(self, instance):
        raise AttributeError('no')


In [ ]:
class ConsDataMixin:
    PI = Constant(3.1415)
    e = Constant(2.71828)

In [ ]:
class MathMeta(type, ConsDataMixin):
    pass

In [ ]:
class Math(ConsDataMixin, metaclass=MathMeta):
    pass

In [ ]:
Math.PI

In [ ]:
Math.PI = 123

In [ ]:
math = Math()

In [ ]:
math.PI

In [ ]:
math.PI = 123

`__getattribute__`, `__getattr__`, `__setattr__`